In [20]:
# Install Java, Spark, and Findspark
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!apt-get update
!apt-get install openjdk-8-jdk
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (124 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [21]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-15 22:56:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.72MB/s    in 0.2s    

2020-08-15 22:56:53 (4.72 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [22]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Big-Data-Project1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [23]:
#%pyspark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [24]:
# the number of records in the data set
print(video_games_df.count())

1785997


In [ ]:
# Create Products DF design for table products
products_df = video_games_df.select("product_id", "product_title").dropDuplicates()
products_df = products_df.distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [ ]:

#look at the schema of products_df
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# Create Review df for table review_id
review_df = video_games_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date").dropDuplicates()
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R25HN3SIQZIUWF|   19145185|B00MV7KVP4|      92208606| 2015-08-31|
|R1DB0RXBWJTSZA|   17033222|B00BU3ZLJQ|     860676261| 2015-08-31|
|R17XE0M8S59SQ8|   10589108|B000069TDF|      13629273| 2015-08-31|
|R1MK4ENBTCKHCG|    2498811|B00VETEZ34|      53980133| 2015-08-31|
|R3VC0HCN87EN7F|   22788662|B00ZSJUTCM|     838780144| 2015-08-31|
|R1TEKXJZIWTFNF|   44777724|B00MV7KVP4|      92208606| 2015-08-30|
|R3J78BD5M97ZHB|   25766524|B00EZV6HHU|     899419151| 2015-08-29|
|R140YF99L4RFAI|   30351861|B000P46NMK|     482320488| 2015-08-29|
|R3NLEZBP3TUUGZ|   49915281|B00KM66UFQ|     703572787| 2015-08-29|
| RRK4AUSOSHZD6|    3887193|B00KVP78FE|     177244653| 2015-08-29|
|R2VRSUZ3YSEU2X|   33553576|B00OM6SAK0|      47086315| 2015-08-29|
|R22UAQR1AFO4QR|   41708426|B009S4JTTC|     756644566| 2015-08

In [ ]:
#look at the schema of review_df
from pyspark.sql.types import *
review_df = review_df.withColumn("customer_id2", review_df["customer_id"].cast(IntegerType())).drop("customer_id").withColumnRenamed("customer_id2", "customer_id")
review_df = review_df.withColumn("product_parent2", review_df["product_parent"].cast(IntegerType())).drop("product_parent").withColumnRenamed("product_parent2", "product_parent")
review_df = review_df.withColumn("review_date2", review_df["review_date"].cast(DateType())).drop("review_date").withColumnRenamed("review_date2", "review_date")
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
customers_df = review_df.groupBy("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52563543|             5|
|   30491638|             2|
|   50333821|             3|
|   23713465|             1|
|   12852151|             4|
|   12907964|             1|
|   52832932|             1|
|   48055567|             1|
|   11304938|             1|
|   36826364|             1|
|   10724241|             1|
|   45801066|             5|
|   33037339|             1|
|    2459426|             1|
|   20074514|             1|
|   52211662|             1|
|   50103623|             2|
|   36969253|             2|
|   45518338|             1|
|   11322220|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
vines_df = video_games_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine").dropDuplicates()
vines_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTHHQM6RSDQK|          4|            0|          1|   N|
|R26I4OP3M9BC2K|          5|            0|          0|   N|
|R2P5HQ648QPWVG|          5|            0|          0|   N|
|R293SZSOOV2DMC|          5|            0|          0|   N|
|R3793DQNKX2INH|          5|            0|          0|   N|
|R1IFTWQW79OCWE|          5|            0|          0|   N|
|R11K7B8ZVWT5J2|          5|            0|          0|   N|
|R1LE118TIRBPKA|          5|            0|          1|   N|
| RB9GZU9BZNUEM|          5|            0|          0|   N|
|R2RO1I6G4TOSYD|          4|            0|          1|   N|
| RXRUAMUV19IY8|          1|            0|          0|   N|
|R2F84MTDEMY5E3|          5|            0|          0|   N|
|R18XB0G8PFU91B|          1|            3|          7|   N|
|R3FVPRCTKXBJO0|          5|            

In [ ]:
vines_df = vines_df.withColumn("star_rating2", vines_df["star_rating"].cast(IntegerType())).drop("star_rating").withColumnRenamed("star_rating2", "star_rating")
vines_df = vines_df.withColumn("helpful_votes2", vines_df["helpful_votes"].cast(IntegerType())).drop("helpful_votes").withColumnRenamed("helpful_votes2", "helpful_votes")
vines_df = vines_df.withColumn("total_votes2", vines_df["total_votes"].cast(IntegerType())).drop("total_votes").withColumnRenamed("total_votes2", "total_votes")
vines_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)



In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://big-data-homework-db.cac8viaacwex.us-east-2.rds.amazonaws.com:5432/amazon_review"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
vines_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)